In [69]:
import mlflow
import pandas as pd

In [46]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [8]:
client.get_experiment('0')

<Experiment: artifact_location='/home/azureuser/MLOps_zoomcamp/notebooks/mlruns/0', creation_time=1685564918509, experiment_id='0', last_update_time=1685564918509, lifecycle_stage='active', name='Default', tags={}>

In [11]:
client.create_experiment(name="project-experiment")

'2'

In [47]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse > 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [52]:
run

<Run: data=<RunData: metrics={'rmse': 8.239120596247158}, params={'learning_rate': '0.15067400192366745',
 'max_depth': '81',
 'min_child_weight': '0.6161995993252231',
 'objective': 'reg:linear',
 'reg_alpha': '0.2870724406903693',
 'reg_lambda': '0.023563243579993714',
 'seed': '42'}, tags={'mlflow.runName': 'wistful-whale-750',
 'mlflow.source.name': '/home/azureuser/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'azureuser',
 'model': 'xgboost'}>, info=<RunInfo: artifact_uri='/home/azureuser/MLOps_zoomcamp/notebooks/mlruns/1/3a78bedb3e754bcd91b9cd55cbf9b1fa/artifacts', end_time=1685566495400, experiment_id='1', lifecycle_stage='active', run_id='3a78bedb3e754bcd91b9cd55cbf9b1fa', run_name='wistful-whale-750', run_uuid='3a78bedb3e754bcd91b9cd55cbf9b1fa', start_time=1685566481467, status='FINISHED', user_id='azureuser'>>

In [48]:
for run in runs:
    print(f"run id: {run.info.run_id}, status:{run.info.status}, rmse: {run.data.metrics['rmse']:.4f}")

run id: db84e9beb0344770a4a98c5d95db9178, status:FINISHED, rmse: 7.7587
run id: d15b2895cd294d99bb19c3490c776ba0, status:FINISHED, rmse: 7.7587
run id: e9e0edd823e34ec2991ad75bcc463cbf, status:FINISHED, rmse: 7.7587
run id: 8ca3efb074af4b3ca79cf415ec78bb82, status:FINISHED, rmse: 8.1343
run id: 3a78bedb3e754bcd91b9cd55cbf9b1fa, status:FINISHED, rmse: 8.2391


In [78]:
run_id = "6bd3ebb0fe124d3f95c09bd65203dae8"
model_uri = f"runs:/{run_id}/models"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-models")

Registered model 'nyc-taxi-models' already exists. Creating a new version of this model...


MissingConfigException: Yaml file '/home/azureuser/MLOps_zoomcamp/notebooks/mlruns/1/6bd3ebb0fe124d3f95c09bd65203dae8/meta.yaml' does not exist.

In [39]:
model_uri

'runs:/d5cd154e343d44918b9ac41b2f647f5a/model'

In [62]:
model_name = "nyc-taxi-models"
latest_version = client.get_latest_versions(name=model_name)

for version in latest_version:
    print(f"version: {version.version}, stage:{version.current_stage}")

version: 1, stage:Archived
version: 2, stage:Production
version: 3, stage:Staging
version: 4, stage:None


In [63]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage='Staging',
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685667773771, current_stage='Staging', description='', last_updated_timestamp=1685671634019, name='nyc-taxi-models', run_id='6bd3ebb0fe124d3f95c09bd65203dae8', run_link='', source='/home/azureuser/MLOps_zoomcamp/notebooks/mlruns/1/6bd3ebb0fe124d3f95c09bd65203dae8/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [64]:
#client.update_model_version

In [73]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [75]:
df = read_dataframe('data/green_tripdata_2021-03.parquet')

In [81]:
mlflow.artifacts.download_artifacts(run_id='6bd3ebb0fe124d3f95c09bd65203dae8', dst_path='.')

MissingConfigException: Yaml file '/home/azureuser/MLOps_zoomcamp/notebooks/mlruns/1/6bd3ebb0fe124d3f95c09bd65203dae8/meta.yaml' does not exist.